# Notebook 04: MCP (Model Context Protocol)

## 🎯 What is This Notebook About?

Welcome to Notebook 04! In this notebook, we'll explore **MCP (Model Context Protocol)** - a protocol for integrating external tools and services with LLMs.

**What we'll learn:**
1. **What is MCP** - Understanding the Model Context Protocol
2. **Tool Execution** - How agents call and use tools
3. **Tool Integration Patterns** - Different ways to integrate tools
4. **Creating Custom Tools** - Building your own tools for agents

**Why this matters:**
- LLMs can't directly interact with systems
- MCP provides a standardized way to connect tools
- Enables agents to take real actions
- Makes agents more powerful and useful

---

## 📚 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what MCP is and why it's important
- ✅ Know how tools are executed by agents
- ✅ Learn different tool integration patterns
- ✅ Be able to create custom tools
- ✅ Understand how tools enable agents to take actions

---

## ⚙️ Prerequisites

- LlamaStack server running (see Module README)
- Ollama running with llama3.2:3b model
- Python environment with dependencies installed
- Understanding of Notebook 03 (Simple Chat and RAG)

---

## 🔧 Setup

Let's start by connecting to LlamaStack and verifying everything is working.


In [ ]:
# Import required libraries
import os
import subprocess
import urllib3
from llama_stack_client import LlamaStackClient

# Suppress SSL warnings for OpenShift self-signed certificates
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_llamastack_url() -> str:
    """Get LlamaStack URL from environment or OpenShift route"""
    # Check environment variable first
    url = os.getenv("LLAMA_STACK_URL")
    if url:
        return url
    
    # Try to get from OpenShift route
    namespace = os.getenv("NAMESPACE", "my-first-model")
    
    try:
        # Method 1: Try by name (most reliable)
        result = subprocess.run(
            ["oc", "get", "route", "llamastack-route", "-n", namespace,
             "-o", "jsonpath={.spec.host}"],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0 and result.stdout:
            return f"https://{result.stdout.strip()}"
        
        # Method 2: Try by label app=llama-stack (with hyphen)
        result = subprocess.run(
            ["oc", "get", "route", "-n", namespace, "-l", "app=llama-stack",
             "-o", "jsonpath={.items[0].spec.host}"],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0 and result.stdout:
            return f"https://{result.stdout.strip()}"
        
        # Method 3: Try by label app=llamastack (without hyphen)
        result = subprocess.run(
            ["oc", "get", "route", "-n", namespace, "-l", "app=llamastack",
             "-o", "jsonpath={.items[0].spec.host}"],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0 and result.stdout:
            return f"https://{result.stdout.strip()}"
    except Exception:
        pass
    
    # Fallback to localhost
    return "http://localhost:8321"

# Configuration
llamastack_url = get_llamastack_url()
model = os.getenv("LLAMA_MODEL", "vllm-inference/llama-32-3b-instruct")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
# Note: For OpenShift routes with self-signed certs, you may need to disable SSL verification
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {len(models)}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("   Please ensure:")
    print("   1. You're logged into OpenShift: oc login")
    print("   2. LlamaStack is deployed in OpenShift")
    print("   3. Or set LLAMA_STACK_URL environment variable")
    print("   4. For OpenShift routes, SSL verification may need to be disabled")
    raise


## Part 1: What is MCP?

**What we're doing:** Understanding MCP - the protocol that lets agents interact with external systems.

**Why:** LLMs are just text processors - they can't directly interact with systems. MCP provides a standardized way to connect tools, so agents can actually DO things, not just talk about them!

### What is MCP?

**MCP (Model Context Protocol)** is a protocol for integrating external tools and services with LLMs. It allows agents to:
- **Call external APIs** (e.g., check service status, restart services)
- **Access databases** (e.g., query incident logs)
- **Execute commands** (e.g., run system commands)
- **Integrate with other systems** (e.g., monitoring tools, ticketing systems)

**Why MCP matters:**
- LLMs can't directly interact with systems (they're text processors, not system operators!)
- MCP provides a standardized way to connect tools (one protocol, many tools)
- Enables agents to take real actions (check status, restart services, query databases)
- Makes agents more powerful and useful (from assistant to operator!)

**When to use MCP:**
- ✅ Need to interact with external systems (monitoring, ticketing, databases)
- ✅ Want agents to take actions (not just answer questions)
- ✅ Need real-time data from APIs (current status, metrics, alerts)
- ✅ Want to integrate with existing tools (your infrastructure, your way!)

---

### Hands-on: Exploring Tool Runtime

Let's explore what tools are available and how they work.


In [ ]:
# Example 1: Understanding MCP Architecture
print("💡 MCP (Model Context Protocol) Components:")
print("\n1. MCP Server:")
print("   - Exposes tools, resources, and prompts")
print("   - Runs as a separate service")
print("   - Communicates via MCP protocol (JSON-RPC 2.0)")
print("   - Examples: MongoDB MCP server, Filesystem MCP server")
print("\n2. MCP Client (LlamaStack):")
print("   - Connects to MCP servers")
print("   - Makes tools available to agents")
print("   - Manages tool execution")
print("\n3. Agent:")
print("   - Uses tools exposed by MCP servers")
print("   - Decides when to call tools")
print("   - Uses tool results in responses")
print("\n💡 MCP Transport Types:")
print("   - stdio: For local scripts (command-line)")
print("   - streamable-http: For web deployments (recommended)")
print("   - sse: Server-Sent Events (legacy)")
print("\n✅ In this notebook, we'll see MongoDB MCP integration in action!")


## Part 2: Understanding Tool Execution

**What we're doing:** Understanding how agents execute tools - the decision-making process that turns thinking into action.

**Why:** Tools are what give agents the ability to take actions. Understanding how agents decide which tools to use helps you build better tools and debug agent behavior.

Tools are functions that agents can call. When an agent needs to perform an action, it:
1. **Decides** which tool to use (reasoning: "I need to check service status")
2. **Calls** the tool with appropriate parameters (execution: `check_service_status("nginx")`)
3. **Receives** the result (response: "Service 'nginx' is running")
4. **Uses** the result to continue reasoning (thinking: "Service is running, so the problem must be elsewhere")

**Tool Structure:**
- **Name**: Identifies the tool (`check_service_status`)
- **Description**: Tells the LLM what the tool does ("Check the status of a system service")
- **Parameters**: What inputs the tool needs (`service_name: str`)
- **Returns**: What the tool outputs (`str` - status message)


In [ ]:
# Example 2: Register MongoDB MCP Server
import requests
import json
import os
import subprocess

def get_mcp_server_url() -> str:
    """Get MCP server URL - use service URL for in-cluster access"""
    # Check environment variable first
    url = os.getenv("MCP_SERVER_URL")
    if url:
        return url
    
    # For OpenShift, use internal service URL (LlamaStack runs in-cluster)
    namespace = os.getenv("NAMESPACE", "my-first-model")
    return f"http://mongodb-mcp-server.{namespace}.svc.cluster.local:3000"

# Get URLs
llamastack_url = get_llamastack_url()  # Use function from cell 1
mcp_server_url = get_mcp_server_url()
toolgroup_id = "mcp::mongodb"

print(f"\n📋 Registration Details:")
print(f"   LlamaStack URL: {llamastack_url}")
print(f"   MCP Server URL: {mcp_server_url}")
print(f"   Toolgroup ID: {toolgroup_id}")
print(f"   Endpoint: {mcp_server_url}/mcp")

# Registration payload
registration_payload = {
    "provider_id": "model-context-protocol",
    "toolgroup_id": toolgroup_id,
    "mcp_endpoint": {
        "uri": f"{mcp_server_url}/mcp"
    }
}

print(f"\n📝 Registration Payload:")
print(json.dumps(registration_payload, indent=2))

print(f"\n💡 To register, make a POST request:")
print(f"   POST {llamastack_url}/v1/toolgroups")
print(f"   Body: {json.dumps(registration_payload, indent=2)}")

# Check if toolgroup already exists
try:
    response = requests.get(
        f"{llamastack_url}/v1/toolgroups/{toolgroup_id}",
        timeout=5,
        verify=False  # OpenShift routes may have self-signed certs
    )
    if response.status_code == 200:
        print(f"\n✅ Toolgroup already registered!")
        toolgroup_data = response.json()
        endpoint = toolgroup_data.get('mcp_endpoint', {}).get('uri', 'N/A')
        print(f"   Endpoint: {endpoint}")
        if endpoint != f"{mcp_server_url}/mcp":
            print(f"   ⚠️  Endpoint differs from expected: {mcp_server_url}/mcp")
    else:
        print(f"\n⚠️  Toolgroup not found (status: {response.status_code})")
        print(f"   You can register it using the code above (Example 2)")
        print(f"   Or make a POST request to {llamastack_url}/v1/toolgroups")
        print(f"   with the registration payload shown above")
except Exception as e:
    print(f"\n⚠️  Could not check toolgroup status: {e}")
    print(f"   Make sure LlamaStack is running and accessible")

print(f"\n💡 Key Points:")
print(f"   - Endpoint must be '/mcp' for streamable-http transport")
print(f"   - Use service URL (not route) when LlamaStack runs in-cluster")
print(f"   - Toolgroup ID format: mcp::<name>")


## Part 3: Creating an MCP Server for Terminal Access

**What we're doing:** Building a practical MCP server that gives agents terminal access - watch them execute commands and interact with your systems!

**Why:** This demonstrates real-world MCP usage. You'll see how to build tools that agents can use, with proper security and validation.

**What we'll build:**
- A simple MCP server that can execute terminal commands (safe commands only!)
- Safe command execution with basic validation (whitelist approach for security)
- Integration with LlamaStack to use terminal tools in agents (connect tools to agents)

**Why this is useful:**
- Agents can execute system commands (check status, view logs, etc.)
- Useful for IT operations tasks (monitoring, troubleshooting, management)
- Demonstrates real-world MCP usage (the pattern you'll use for production tools)

**The fun part:** You'll see agents execute commands like `ps`, `df`, `uptime` - watch them gather system information and use it to make decisions!


In [ ]:
# Example 3: Create Agent with MongoDB MCP Tools
# Agent configuration with MCP tools (using dictionary - more compatible)
agent_config = {
    "model": "vllm-inference/llama-32-3b-instruct",  # or "ollama/llama3.2:3b" for local
    "instructions": (
        "You are a helpful assistant with access to MongoDB through MCP tools. "
        "When asked about databases or data, you MUST use the MongoDB MCP tools to query the database. "
        "Always use the tools to get real data from MongoDB."
    ),
    "toolgroups": ["mcp::mongodb"],  # Include MongoDB MCP toolgroup
    "tool_choice": "auto",  # Let agent decide when to use tools
    "sampling_params": {
        "max_tokens": 2000  # Must be > 0!
    }
}

print("\n📋 Agent Configuration:")
print(f"   Model: {agent_config['model']}")
print(f"   Toolgroups: {agent_config['toolgroups']}")
print(f"   Tool Choice: {agent_config['tool_choice']}")
print(f"   Max Tokens: {agent_config['sampling_params'].get('max_tokens', 'not set')}")

print("\n💡 Key Points:")
print("   - toolgroups must include 'mcp::mongodb'")
print("   - tool_choice='auto' lets agent decide when to use tools")
print("   - max_tokens must be > 0 (default is 0, which causes errors)")

# Create agent
print("\n🚀 Creating agent...")
try:
    # Try client.agents.create first (newer API)
    try:
        agent = client.agents.create(agent_config=agent_config)
    except AttributeError:
        # Fall back to alpha API if needed
        agent = client.alpha.agents.create(agent_config=agent_config)
    
    print(f"\n✅ Agent created successfully!")
    print(f"   Agent ID: {agent.agent_id}")
    print(f"\n💡 This agent can now use MongoDB MCP tools!")
    print(f"   In the next example, we'll query it and see it use the tools.")
    
except Exception as e:
    print(f"\n❌ Error creating agent: {e}")
    print("   Make sure:")
    print("   1. LlamaStack is running")
    print("   2. MongoDB MCP toolgroup is registered (see Example 2)")
    print("   3. Model is available")
    print(f"\n   You can still continue with the conceptual examples below.")
    agent = None


## Part 4: How Agents Use MCP Tools

**What we're doing:** Understanding the flow of how agents call MCP tools and use the results.

**Why:** This shows the complete interaction cycle - from user query to tool execution to final response.

### Tool Execution Flow

```
User Query
    ↓
Agent receives query
    ↓
Agent decides to use tool (reasoning)
    ↓
Agent calls MCP tool (e.g., list-databases)
    ↓
LlamaStack forwards to MCP server
    ↓
MCP server executes tool (queries MongoDB)
    ↓
MCP server returns results
    ↓
Agent receives tool results
    ↓
Agent generates response using real data
```

**Key Points:**
- Agent decides autonomously when to use tools
- Tool calls are visible in the streaming response
- Tool results are included in the agent's context
- Agent uses real data to generate accurate responses


In [ ]:
# Example 4: Query Agent with MongoDB Tools
# This example shows how to interact with an agent that has MCP tools
# The agent will automatically use MongoDB tools when needed

print("💡 Example Agent Interaction Flow:\n")

scenarios = [
    {
        "user_query": "What databases are available in MongoDB?",
        "agent_reasoning": "I need to check what databases exist. I'll use the list-databases tool.",
        "tool_call": {
            "tool": "mcp::mongodb::list-databases",
            "arguments": {}
        },
        "tool_result": {
            "databases": ["admin", "config", "local", "mcp_demo"]
        },
        "agent_response": "There are 4 databases available in MongoDB: admin, config, local, and mcp_demo."
    },
    {
        "user_query": "What collections are in the mcp_demo database?",
        "agent_reasoning": "I need to list collections in the mcp_demo database. I'll use the list-collections tool.",
        "tool_call": {
            "tool": "mcp::mongodb::list-collections",
            "arguments": {"database": "mcp_demo"}
        },
        "tool_result": {
            "collections": ["incidents", "users", "logs"]
        },
        "agent_response": "The mcp_demo database contains 3 collections: incidents, users, and logs."
    },
    {
        "user_query": "How many incidents are in the incidents collection?",
        "agent_reasoning": "I need to count documents in the incidents collection. I'll use the count-documents tool.",
        "tool_call": {
            "tool": "mcp::mongodb::count-documents",
            "arguments": {"database": "mcp_demo", "collection": "incidents"}
        },
        "tool_result": {
            "count": 42
        },
        "agent_response": "There are 42 incidents in the incidents collection."
    }
]

for i, scenario in enumerate(scenarios, 1):
    print(f"{'='*60}")
    print(f"Scenario {i}: {scenario['user_query']}")
    print(f"{'='*60}")
    print(f"\n👤 User: {scenario['user_query']}")
    print(f"\n🤖 Agent Reasoning: {scenario['agent_reasoning']}")
    print(f"\n🔧 Tool Call:")
    print(f"   Tool: {scenario['tool_call']['tool']}")
    print(f"   Arguments: {json.dumps(scenario['tool_call']['arguments'], indent=2)}")
    print(f"\n📤 Tool Result:")
    print(f"   {json.dumps(scenario['tool_result'], indent=2)}")
    print(f"\n💬 Agent Response: {scenario['agent_response']}")
    print()

print("\n✅ This demonstrates how agents:")
print("   - Automatically decide when to use tools")
print("   - Call MCP tools with appropriate arguments")
print("   - Use real data from MongoDB")
print("   - Generate accurate responses based on tool results")


## Part 5: Complete Integration Workflow

**What we're doing:** Seeing the complete MongoDB MCP integration workflow.

**Why:** This summarizes all the steps needed to integrate MongoDB MCP with LlamaStack agents.

**Complete Workflow:**
1. **Deploy MongoDB MCP Server** (already done in OpenShift)
2. **Register MCP Server** with LlamaStack (Example 2)
3. **Create Agent** with MongoDB toolgroup (Example 3)
4. **Query Agent** - it uses MongoDB tools automatically (Example 6)
5. **View Results** - see tool calls and real data


In [ ]:
# Example 5: Complete MongoDB MCP Integration Example
# This shows the complete workflow - all steps are demonstrated in the examples above

steps = [
    {
        "step": "1. Deploy MongoDB MCP Server",
        "description": "MCP server runs as a separate service in OpenShift",
        "command": "oc apply -f openshift/manifests/mongodb/mongodb-mcp-server-deployment.yaml",
        "details": "Server exposes MongoDB operations as MCP tools"
    },
    {
        "step": "2. Register MCP Server with LlamaStack",
        "description": "Create toolgroup so LlamaStack knows about the MCP server",
        "code": """
# Use the registration code from Example 2 above
# Or make a POST request:
import requests
response = requests.post(
    f"{llamastack_url}/v1/toolgroups",
    json={
        "provider_id": "model-context-protocol",
        "toolgroup_id": "mcp::mongodb",
        "mcp_endpoint": {
            "uri": "http://mongodb-mcp-server.<namespace>.svc.cluster.local:3000/mcp"
        }
    },
    verify=False
)
        """,
        "details": "Registers 'mcp::mongodb' toolgroup with endpoint '/mcp'"
    },
    {
        "step": "3. Create Agent with MongoDB Tools",
        "description": "Agent must include toolgroup in configuration",
        "code": """
agent_config = {
    "model": "vllm-inference/llama-32-3b-instruct",
    "instructions": "You have access to MongoDB through MCP tools.",
    "toolgroups": ["mcp::mongodb"],  # Include MongoDB toolgroup
    "tool_choice": "auto",
    "sampling_params": {"max_tokens": 2000}
}
# Try client.agents.create first, or client.alpha.agents.create
agent = client.agents.create(agent_config=agent_config)
        """,
        "details": "Agent can now use MongoDB MCP tools"
    },
    {
        "step": "4. Query Agent",
        "description": "Agent automatically uses MongoDB tools when needed",
        "code": """
# Create session
session = client.agents.session.create(agent_id=agent.agent_id)

# Create turn (with streaming)
turn = client.agents.turn.create(
    agent_id=agent.agent_id,
    session_id=session.session_id,
    messages=[{"role": "user", "content": "What databases are in MongoDB?"}],
    stream=True
)
        """,
        "details": "Agent calls MongoDB tools and uses results in response"
    }
]

print("📋 Complete MongoDB MCP Integration Workflow:\n")

for step_info in steps:
    print(f"{step_info['step']}")
    print(f"   {step_info['description']}")
    if 'command' in step_info:
        print(f"   Command: {step_info['command']}")
    if 'code' in step_info:
        print(f"   Code:")
        for line in step_info['code'].strip().split('\n'):
            print(f"      {line}")
    print(f"   Details: {step_info['details']}")
    print()

print("💡 Key Learnings:")
print("   - MCP server endpoint must be '/mcp' for streamable-http transport")
print("   - Use service URL (not route) when LlamaStack runs in-cluster")
print("   - Agent must include toolgroup in 'toolgroups' array")
print("   - max_tokens must be > 0 in agent configuration")
print("   - Agent decides autonomously when to use tools")


## Part 6: Testing the Integration

**What we're doing:** Testing the complete MongoDB MCP integration to verify everything works.

**Why:** It's important to verify that the MCP server is registered, tools are available, and agents can use them.

### Testing Steps

1. **Verify toolgroup is registered** - Check that MongoDB MCP toolgroup exists
2. **List available toolgroups** - See all registered toolgroups
3. **Create and query agent** - Actually use the agent with MongoDB tools (if agent was created in Example 3)


In [ ]:
# Example 6: Test MongoDB MCP Integration
# Use the llamastack_url from cell 1 (or get it again if needed)
test_llamastack_url = llamastack_url if 'llamastack_url' in globals() else get_llamastack_url()

# Step 1: Check if toolgroup is registered
print("Step 1: Verify Toolgroup Registration")
try:
    response = requests.get(
        f"{test_llamastack_url}/v1/toolgroups/mcp::mongodb",
        timeout=5,
        verify=False
    )
    if response.status_code == 200:
        toolgroup_data = response.json()
        print(f"   ✅ Toolgroup 'mcp::mongodb' is registered")
        endpoint = toolgroup_data.get('mcp_endpoint', {}).get('uri', 'N/A')
        print(f"   Endpoint: {endpoint}")
    else:
        print(f"   ⚠️  Toolgroup not found (status: {response.status_code})")
        print(f"   Register it using Example 2 code above")
except Exception as e:
    print(f"   ❌ Error checking toolgroup: {e}")

print()

# Step 2: List available toolgroups
print("Step 2: List Available Toolgroups")
try:
    response = requests.get(
        f"{test_llamastack_url}/v1/toolgroups",
        timeout=5,
        verify=False
    )
    if response.status_code == 200:
        toolgroups = response.json()
        if isinstance(toolgroups, list):
            toolgroups = toolgroups
        elif isinstance(toolgroups, dict) and 'data' in toolgroups:
            toolgroups = toolgroups['data']
        else:
            toolgroups = []
        
        print(f"   ✅ Found {len(toolgroups)} toolgroup(s):")
        for tg in toolgroups:
            tg_id = tg.get('identifier', tg.get('toolgroup_id', 'unknown'))
            provider = tg.get('provider_id', 'unknown')
            print(f"      - {tg_id} ({provider})")
    else:
        print(f"   ⚠️  Could not list toolgroups (status: {response.status_code})")
except Exception as e:
    print(f"   ❌ Error listing toolgroups: {e}")

print()

# Step 3: Create and Query Agent (if agent was created in Example 3)
if 'agent' in globals() and agent is not None:
    print("Step 3: Query Agent with MongoDB Tools")
    print("   Creating session and querying agent...\n")
    
    try:
        # Create session
        try:
            session = client.agents.session.create(agent_id=agent.agent_id, session_name="mongodb-test")
        except AttributeError:
            session = client.alpha.agents.session.create(agent_id=agent.agent_id, session_name="mongodb-test")
        
        print(f"✅ Session created: {session.session_id}\n")
        
        # Query agent
        query = "What databases are available in MongoDB?"
        print(f"👤 Query: {query}\n")
        
        try:
            turn_stream = client.agents.turn.create(
                agent_id=agent.agent_id,
                session_id=session.session_id,
                messages=[{"role": "user", "content": query}],
                stream=True
            )
        except AttributeError:
            turn_stream = client.alpha.agents.turn.create(
                agent_id=agent.agent_id,
                session_id=session.session_id,
                messages=[{"role": "user", "content": query}],
                stream=True
            )
        
        def to_dict(obj):
            """Convert object to dictionary"""
            if isinstance(obj, dict):
                return obj
            if hasattr(obj, 'model_dump'):
                return obj.model_dump()
            if hasattr(obj, 'dict'):
                return obj.dict()
            if hasattr(obj, '__dict__'):
                return {k: v for k, v in obj.__dict__.items() if not k.startswith('_')}
            return {}
        
        # Get turn_id (exactly like test script)
        turn_id = None
        for chunk in turn_stream:
            if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
                payload = chunk.event.payload
                d = to_dict(payload)
                turn_id = d.get('turn_id')
                if not turn_id and 'step_details' in d:
                    step_details = to_dict(d['step_details'])
                    turn_id = step_details.get('turn_id')
                if turn_id:
                    break
        
        if turn_id:
            print("⏳ Waiting for response...")
            import time
            import httpx
            time.sleep(2)
            response = httpx.get(
                f"{test_llamastack_url}/v1alpha/agents/{agent.agent_id}/session/{session.session_id}/turn/{turn_id}",
                verify=False, timeout=30
            )
            response.raise_for_status()
            
            # Handle null response
            if not response.text or response.text.strip() == 'null':
                print("   ⚠️  Turn not ready yet (response was null)")
                print("   💡 The turn may still be processing. Try waiting a bit longer.")
                data = None
            else:
                data = response.json()
            
            if data is None:
                print("   ⚠️  Could not retrieve turn data")
                print("   💡 This might mean the turn is still processing or the turn_id is incorrect")
            else:
                # Get response from messages or steps
                messages = data.get('messages', [])
                steps = data.get('steps', [])
                response_text = None
            
                # Debug: Show what we found
                print("=" * 60)
                print("Debug Information")
                print("=" * 60)
                print(f"Messages: {len(messages)}")
                print(f"Steps: {len(steps)}\n")
            
                # Show step details
                if steps:
                    print("Steps found:")
                for i, step in enumerate(steps):
                    step_type = step.get('type', '') or step.get('step_type', '')
                    print(f"  Step {i}: type='{step_type}'")
                    
                    # Show tool calls
                    if step.get('type') == 'tool_call' or step_type == 'tool_call':
                        tool_call = step.get('tool_call', {})
                        tool_name = tool_call.get('name', 'unknown')
                        tool_args = tool_call.get('arguments', {})
                        print(f"    Tool: {tool_name}")
                        if tool_args:
                            args_str = str(tool_args)[:100]
                            print(f"    Args: {args_str}")
                    
                    # Show tool results
                    if step.get('type') == 'tool_result' or step_type == 'tool_result':
                        tool_result = step.get('tool_result', {})
                        tool_name = tool_result.get('name', 'unknown')
                        print(f"    Tool Result: {tool_name}")
                
                print()
            
                # Try messages first
                for msg in reversed(messages):
                    if msg.get('role') == 'assistant':
                        content = msg.get('content', '')
                    if isinstance(content, str) and content.strip():
                        response_text = content
                        break
                    elif isinstance(content, list) and content:
                        for item in content:
                            if isinstance(item, dict):
                                text = item.get('text', '')
                                if text:
                                    response_text = text
                                    break
                            elif isinstance(item, str) and item.strip():
                                response_text = item
                                break
                        if response_text:
                            break
            
                # If no response in messages, check steps
                if not response_text:
                for step in reversed(steps):
                    # Try inference step
                    if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                        model_response = step.get('model_response', {})
                        content = model_response.get('content', '')
                        if isinstance(content, str) and content.strip():
                            response_text = content
                            break
                    
                    # Try any step with model_response
                    if 'model_response' in step:
                        model_response = step.get('model_response', {})
                        content = model_response.get('content', '')
                        if isinstance(content, str) and content.strip():
                            response_text = content
                            break
            
            # Display final response
                print("=" * 60)
                print("Agent Response")
                print("=" * 60)
            
                if response_text:
                    print(f"\n💬 {response_text}\n")
                
                # Show tools used
                tools = []
                for step in steps:
                    if step.get('type') == 'tool_call' or step.get('step_type') == 'tool_call':
                        tool_call = step.get('tool_call', {})
                        tool_name = tool_call.get('name', 'unknown')
                        if tool_name not in tools:
                            tools.append(tool_name)
                
                if tools:
                    print(f"🔧 Tools used: {', '.join(tools)}\n")
                else:
                    print(f"\n⚠️  No response found\n")
        else:
                print("❌ Could not get turn_id")
        
                print("\n✅ Agent successfully queried MongoDB using MCP tools!")
        
                except Exception as e:
                print(f"❌ Error querying agent: {e}")
                import traceback
                traceback.print_exc()
                print("\n💡 You can still see the conceptual examples in Example 4")
else:
                print("Step 3: Create and Query Agent")
                print("   ⚠️  Agent not created (see Example 3 above)")
                print("   Once you create an agent, you can query it like this:")
                print("   1. Create session: session = client.agents.session.create(agent_id=agent.agent_id)")
                print("   2. Query agent: turn = client.agents.turn.create(..., stream=True)")
                print("   3. Process streaming response to see tool calls and results")

                print("\n✅ Integration testing complete!")


---

## 🎓 Key Takeaways

**What we learned:**

1. **MCP (Model Context Protocol)** provides a standardized protocol for tool integration - one protocol, many tools!
2. **Tools** enable agents to take actions, not just answer questions - from thinking to doing!
3. **Tool execution** follows a clear pattern: decide → call → receive → use - watch the agent reason and act!
4. **MCP servers** can provide terminal access and other capabilities - connect agents to YOUR systems!
5. **Security** is important - use whitelists and validation for command execution (safety first!)

**The big picture:**
- **MCP** standardizes tool integration - build tools once, use with any agent
- **Tools** give agents "hands" - they can interact with external systems
- **Tool execution** is transparent - you can see what the agent is doing
- **Security** is critical - always validate inputs and use whitelists

**For IT operations:**
- Build MCP tools for your monitoring systems, ticketing systems, databases
- Give agents access to terminal commands (safely!) for system management
- Connect agents to your APIs and services
- Create production-ready tools with proper security and validation

**Security considerations:**
- ✅ Use command whitelists (only allow safe commands)
- ✅ Validate all inputs (never trust user input!)
- ✅ Set timeouts on command execution (prevent hanging)
- ✅ Run with minimal privileges (principle of least privilege)
- ✅ Log all command executions (audit trail)
- ✅ Consider sandboxing for production (isolate execution)

---

## 🚀 Next Steps

**Ready for more?** In **Notebook 05**, we'll explore:
- **Safety Shields** - Content moderation and safety checks (protect against harmful outputs)
- **Content Moderation** - Checking inputs and outputs for safety violations
- **Production-ready safety** - Building agents that are safe to deploy

**The fun part:** You'll learn how to protect your agents from harmful inputs and outputs - essential for production deployment!

---

**Ready?** Let's move to Notebook 05: Safety Shields! 🚀
